In [2]:
import pandas as pd
import numpy as np
import polars as pl
import time
import matplotlib.pyplot as plt

#set path to REMOTES folder
path = '../../../data'

# Load the data
df = pd.read_excel(f'{path}/data_labels.xlsx')

#read in signal freq -> dictionary
file = open(f'{path}/label_freq.txt', 'r')
label_freq = file.read()
label_freq = eval(label_freq)
file.close()

ecg_fs = label_freq['ECG_A']
print(ecg_fs)


244.140625


In [13]:
#set path to REMOTES folder
path = '../../../data'

# Load the data
df = pd.read_csv(f'{path}/sensors_data.csv')

#open and read the text file with patient IDs to remove 
import ast

with open(f'{path}/patients_to_remove2.txt') as f:
    lines = f.read()  # Read the entire content
    patients_to_remove = ast.literal_eval(lines)  # Safely evaluate the string as a list

print(len(patients_to_remove))

#add these to ID's to the patients_to_remove list: ['R013', 'R029', 'R042', 'R078', 'R088', 'R135']
#patients_to_remove.extend(['R013', 'R029', 'R042', 'R078', 'R088', 'R135'])
print(len(patients_to_remove))


df = df[~df['Patient ID'].isin(patients_to_remove)]

print(len(df))

#remove patient with ID R035
df = df[df['Patient ID'] != 'R035']

print(len(df))


27
27
170
169


In [14]:
#open the demo_data file 
demo_data = pd.read_csv(f'{path}/demo_data.csv')

#change the column title from study_id to Patient ID
demo_data = demo_data.rename(columns={'study_id': 'Patient ID'})

#merge the demo_data with the WS_data only keeping columns 'cpet_quality', cpet_bike, vo2peak_measured, anaerobic_threshold, bmi, age, gender
df = pd.merge(df, demo_data[['Patient ID', 'cpet_quality', 'cpet_bike', 'vo2peak_measured', 'anaerobicthreshold', 'bmi', 'gender', 'height', 'age']], on='Patient ID', how='left')

#remove any patients who have a value of 3 in the cpet_quality column
#df = df[df['cpet_quality'] != 3]


#if cpet_bike is 2, multiply vo2peak_measured by 0.9
df['vo2peak_measured'] = np.where(df['cpet_bike'] == 2, df['vo2peak_measured'] * 0.9, df['vo2peak_measured'])


print(len(df))
print(df.columns)

169
Index(['Patient ID', 'Biosensor ID', 'file_name', 'Start', 'Wear time',
       'missing ACC (% of total signal)',
       'missing ECG overall (% of total signal)',
       'Missing ECG from electrode (% of total signal)',
       '%_ECG_acceptable_quality', 'Hours of data collected', 'SDNN24',
       'SDNNhr_1min', 'SDNNhr_5min', 'SDNNhr_10min', 'SDNNhr_30min',
       'SDNNhr_1hr', 'SDNNhr_10s', 'RMSSD', 'SDNN', 'pNN50', 'MeanNN', 'LF',
       'VLF', 'HF', 'LF_HF', 'SD1', 'SD2', 'steps', 'MVPA steps', 'Q1', 'Q2',
       'Q3', 'Q95', 'Resting HR', 'Max HR', 'Min HR', 'Time in MVPA',
       'Time in LPA', 'Time in SB', 'MVPA HR', 'LPA HR', 'SB HR',
       'cpet_quality', 'cpet_bike', 'vo2peak_measured', 'anaerobicthreshold',
       'bmi', 'gender', 'height', 'age'],
      dtype='object')


In [11]:
ws = ['steps', 'MVPA steps', 
       'Resting HR', 'Max HR', 'Min HR', 'Time in MVPA', 'Time in LPA',
       'Time in SB', 'SB HR', 
        'Q1', 'Q2', 'Q3', 'Q95', 'pNN50', 'MeanNN','LF', 'HF', 'VLF',
       'LF_HF', 'SD2', 'SDNN24',
       'SDNNhr_1min', 'SDNNhr_5min', 'SDNNhr_10min', 'SDNNhr_30min', 'SDNNhr_1hr',
       'gender', 'bmi', 'age']

print(len(ws))

29


In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Features and target
ws = ['steps', 'MVPA steps', 'Resting HR', 'Max HR', 'Min HR', 'Time in MVPA',
      'Time in LPA', 'Time in SB', 'SB HR', 'Q1', 'Q2', 'Q3', 'Q95', 'pNN50',
      'MeanNN', 'LF', 'HF', 'VLF', 'LF_HF', 'SD2', 'SDNN24', 'SDNNhr_1min',
      'SDNNhr_5min', 'SDNNhr_10min', 'SDNNhr_30min', 'SDNNhr_1hr', 'gender',
      'bmi', 'age']
features_lr = ['steps', 'age', 'gender', 'bmi', 'SDNNhr_1hr', 'LF_HF']

X = df[ws]
y = df['vo2peak_measured']

In [18]:

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize data for models requiring scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Standardize only LASSO-selected features for LR and ML models without feature selection
X_train_lr = X_train[features_lr]
X_val_lr = X_val[features_lr]
X_test_lr = X_test[features_lr]
X_train_lr_scaled = scaler.fit_transform(X_train_lr)
X_val_lr_scaled = scaler.transform(X_val_lr)
X_test_lr_scaled = scaler.transform(X_test_lr)

# Metrics function
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    correlation = np.corrcoef(y_true, y_pred)[0, 1]
    return mae, rmse, r2, correlation

# Results dictionary
results = {}

# 1. Linear Regression
lr = LinearRegression()
lr.fit(X_train_lr_scaled, y_train)
y_pred_lr = lr.predict(X_test_lr_scaled)
results['Linear Regression'] = evaluate_model(y_test, y_pred_lr)

# 2. Random Forest
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
results['Random Forest'] = evaluate_model(y_test, y_pred_rf)

# 3. XGBoost
xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
results['XGBoost'] = evaluate_model(y_test, y_pred_xgb)

# 4. Support Vector Regression (SVR) with parameter tuning
svr = SVR()
param_grid_svr = {'C': [0.1, 1, 10], 'gamma': ['scale', 0.1, 0.01]}
svr_grid = GridSearchCV(svr, param_grid_svr, cv=5, scoring='neg_mean_squared_error')
svr_grid.fit(X_train_lr_scaled, y_train)
best_svr = svr_grid.best_estimator_
y_pred_svr = best_svr.predict(X_test_lr_scaled)
results['SVR'] = evaluate_model(y_test, y_pred_svr)

# 5. Multilayer Perceptron (MLP) with parameter tuning
from sklearn.preprocessing import StandardScaler

# Scale the target variable
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_val_scaled = y_scaler.transform(y_val.values.reshape(-1, 1)).ravel()
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1)).ravel()

# MLP with hyperparameter tuning
mlp = MLPRegressor(max_iter=1000, early_stopping=True, random_state=42)
param_grid_mlp = {
    'hidden_layer_sizes': [(64, 32), (128, 64), (32,)],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01, 0.0001]
}
mlp_grid = GridSearchCV(mlp, param_grid_mlp, cv=5, scoring='neg_mean_squared_error')
mlp_grid.fit(X_train_lr_scaled, y_train_scaled)
best_mlp = mlp_grid.best_estimator_

# Predict and reverse scale
y_pred_mlp_scaled = best_mlp.predict(X_test_lr_scaled).reshape(-1, 1)
y_pred_mlp = y_scaler.inverse_transform(y_pred_mlp_scaled).ravel()

# Evaluate and store results
results['MLP'] = evaluate_model(y_test, y_pred_mlp)

# Display Results
results_df = pd.DataFrame(results, index=['MAE', 'RMSE', 'R²', 'Correlation'])
print(results_df)

/Users/aronsyversen/miniforge3/envs/hr-env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/aronsyversen/miniforge3/envs/hr-env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/aronsyversen/miniforge3/envs/hr-env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/aronsyversen/miniforge3/envs/hr-env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the opti

             Linear Regression  Random Forest   XGBoost       SVR       MLP
MAE                   1.847187       2.274877  2.419041  1.955094  1.745018
RMSE                  2.409853       2.970886  3.133027  2.513777  2.266857
R²                    0.597859       0.388820  0.320287  0.562427  0.644168
Correlation           0.775695       0.645327  0.586521  0.754021  0.804347


In [23]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Metrics function with Average Percentage Error
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    correlation = np.corrcoef(y_true, y_pred)[0, 1]
    ape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # Average percentage error
    return mae, rmse, r2, correlation, ape

# Initialize results storage
results = {'Linear Regression': [], 'Random Forest': [], 'XGBoost': [], 'SVR': [], 'MLP': []}

# Feature selection for Linear Regression and models that need scaling
features_lr = ['steps', 'age', 'gender', 'bmi', 'SDNNhr_1hr', 'LF_HF']

# Loop through each fold
for train_index, test_index in kf.split(X):
    # Split data into train and test for the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Scale data for models that require it
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train_lr = X_train[features_lr]
    X_test_lr = X_test[features_lr]
    X_train_lr_scaled = scaler.fit_transform(X_train_lr)
    X_test_lr_scaled = scaler.transform(X_test_lr)
    
    # Scale target variable for MLP
    y_scaler = StandardScaler()
    y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).ravel()
    y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1)).ravel()
    
    # 1. Linear Regression
    lr = LinearRegression()
    lr.fit(X_train_lr_scaled, y_train)
    y_pred_lr = lr.predict(X_test_lr_scaled)
    results['Linear Regression'].append(evaluate_model(y_test, y_pred_lr))
    
    # 2. Random Forest
    rf = RandomForestRegressor(random_state=42)
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    results['Random Forest'].append(evaluate_model(y_test, y_pred_rf))
    
    # 3. XGBoost
    xgb = XGBRegressor(random_state=42)
    xgb.fit(X_train, y_train)
    y_pred_xgb = xgb.predict(X_test)
    results['XGBoost'].append(evaluate_model(y_test, y_pred_xgb))
    
    # 4. Support Vector Regression (SVR)
    svr = SVR(C=1, gamma='scale')
    svr.fit(X_train_lr_scaled, y_train)
    y_pred_svr = svr.predict(X_test_lr_scaled)
    results['SVR'].append(evaluate_model(y_test, y_pred_svr))
    
    # 5. Multilayer Perceptron (MLP)
    # Define the base MLP model with default parameters
    mlp = MLPRegressor(max_iter=1000, early_stopping=True, random_state=42)

    # Define the hyperparameter grid for tuning
    param_grid_mlp = {
        'hidden_layer_sizes': [(64, 32), (128, 64), (32,)],
        'alpha': [0.0001, 0.001, 0.01],  # Regularization strength
        'learning_rate_init': [0.001, 0.01, 0.0001]  # Learning rate
    }

    # Use GridSearchCV for hyperparameter tuning with 5-fold CV
    mlp_grid = GridSearchCV(
        mlp, 
        param_grid_mlp, 
        cv=5, 
        scoring='neg_mean_squared_error', 
        n_jobs=-1  # Use all available cores for faster tuning
    )
    mlp_grid.fit(X_train_lr_scaled, y_train_scaled)

    # Extract the best model from the grid search
    best_mlp = mlp_grid.best_estimator_

    # Use the best model to predict on the test set
    y_pred_mlp_scaled = best_mlp.predict(X_test_lr_scaled).reshape(-1, 1)

    # Reverse the scaling to get predictions in the original scale of `y`
    y_pred_mlp = y_scaler.inverse_transform(y_pred_mlp_scaled).ravel()

    # Evaluate the best MLP model and append results
    results['MLP'].append(evaluate_model(y_test, y_pred_mlp))

# Average results across all folds
averaged_results = {model: np.mean(metrics, axis=0) for model, metrics in results.items()}

# Display Results
metrics_df = pd.DataFrame(averaged_results, index=['MAE', 'RMSE', 'R²', 'Correlation', 'APE'])
print(metrics_df)


/Users/aronsyversen/miniforge3/envs/hr-env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/aronsyversen/miniforge3/envs/hr-env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/aronsyversen/miniforge3/envs/hr-env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/aronsyversen/miniforge3/envs/hr-env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the opti

             Linear Regression  Random Forest    XGBoost        SVR        MLP
MAE                   2.457099       2.934573   3.289194   2.814690   2.523089
RMSE                  3.230979       3.803573   4.266638   3.756183   3.319725
R²                    0.496897       0.309607   0.148841   0.343466   0.472028
Correlation           0.707125       0.586720   0.490499   0.634814   0.699313
APE                  14.435218      17.535077  19.316523  15.887798  14.745376
